## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

ModuleNotFoundError: No module named 'config'

In [ ]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("/Weather_Database/WeatherPy_database.csv")
city_data_df.head()

In [24]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 85


In [30]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather description
0,0,Padang,ID,-0.9492,100.3543,83.82,73,95,3.76,overcast clouds
2,2,Sarangani,PH,5.4033,125.4636,81.25,78,100,14.38,overcast clouds
6,6,Salalah,OM,17.0151,54.0924,82.49,78,75,3.44,broken clouds
8,8,Coahuayana,MX,18.7333,-103.6833,78.57,92,92,0.34,overcast clouds
9,9,Mineral Wells,US,32.8085,-98.1128,77.92,46,1,8.05,clear sky


In [31]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                189
City                   189
Country                189
Lat                    189
Lng                    189
Max Temp               189
Humidity               189
Cloudiness             189
Wind Speed             189
Weather description    189
dtype: int64

In [32]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df

In [33]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather description,Lat,Lng,Hotel Name
0,Padang,ID,83.82,overcast clouds,-0.9492,100.3543,
2,Sarangani,PH,81.25,overcast clouds,5.4033,125.4636,
6,Salalah,OM,82.49,broken clouds,17.0151,54.0924,
8,Coahuayana,MX,78.57,overcast clouds,18.7333,-103.6833,
9,Mineral Wells,US,77.92,clear sky,32.8085,-98.1128,
17,Port Macquarie,AU,80.56,broken clouds,-31.4333,152.9167,
19,Hambantota,LK,80.94,few clouds,6.1241,81.1185,
20,Amahai,ID,82.62,scattered clouds,-3.3333,128.9167,
21,Bontang,ID,82.51,scattered clouds,0.1333,117.5000,
29,Kapaa,US,82.38,light rain,22.0752,-159.3190,


In [35]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing result")

Missing result
Missing result
Missing result
Missing result
Missing result
Missing result
Missing result
Missing result
Missing result
Missing result
Missing result
Missing result
Missing result


In [42]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   189
Country                189
Max Temp               189
Weather description    189
Lat                    189
Lng                    189
Hotel Name             189
dtype: int64

In [43]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [48]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>Weather description</dt><dd>{Weather description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [49]:
# 11a. Add a marker layer for each city to the map. 

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))